In [2]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('../Data/final_data.csv')

In [4]:
df

,id,title,info
0,19995,Avatar,in the 22nd century a paraplegic marine is di...
1,285,Pirates of the Caribbean: At World's End,captain barbossa long believed to be dead ha...
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,john carter is a war-weary former military ca...
...,...,...,...
4780,9367,El Mariachi,el mariachi just wants to play his guitar and ...
4781,72766,Newlyweds,a newlywed couples honeymoon is upended by the...
4782,231617,"Signed, Sealed, Delivered","""signed sealed delivered"" introduces a dedic..."
4783,126186,Shanghai Calling,when ambitious new york attorney sam is sent t...


In [5]:
import tqdm as notebook_tqdm

In [6]:
from sentence_transformers import SentenceTransformer, util
import torch

/opt/anaconda3/envs/movierecsys/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [8]:
embeddings = model.encode(df['info'].tolist(), convert_to_tensor=True)

In [9]:
embeddings.shape

torch.Size([4785, 384])

In [10]:
df['embeddings'] = embeddings.tolist()


In [11]:
len(df['embeddings'][3])

384

In [12]:
df.loc[df['id'] == 285, 'embeddings']

1    [-0.04783881828188896, -0.07887071371078491, -...
Name: embeddings, dtype: object

In [13]:
def find_similar_movies(movie_id, df, top_n=5):
    # Get the embedding of the selected movie (convert list to tensor)
    query_embedding = torch.tensor(df.loc[df['id'] == movie_id, 'embeddings'].values[0])

    # Convert all embeddings in the DataFrame to a tensor
    all_embeddings = torch.stack([torch.tensor(emb) for emb in df['embeddings'].tolist()])

    # Compute cosine similarity between query movie and all movies
    similarities = util.pytorch_cos_sim(query_embedding, all_embeddings)[0]

    # Get top N similar movies (excluding the query movie itself)
    top_indices = torch.topk(similarities, k=top_n+1).indices.tolist()  # +1 to exclude the query movie itself

    # Extract and print movie details
    print(f"Movies similar to: {df.loc[df['id'] == movie_id, 'title'].values[0]}\n")
    print(f"Description of the current movie: {df.loc[df['id'] == movie_id, 'info'].values[0]}\n")
    for idx in top_indices[1:]:  # Skip the first one since it's the query movie itself
        movie = df.iloc[idx]
        print(f"Title: {movie['title']}")
        print(f"Description: {movie['info']}")
        print(f"Similarity Score: {similarities[idx].item():.4f}")
        print("-" * 50)

# Example usage for movie ID 34
find_similar_movies(19995, df)

Movies similar to: Avatar

Description of the current movie: in the 22nd century  a paraplegic marine is dispatched to the moon pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization. jamescameron action adventure fantasy sciencefiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d ingeniousfilmpartners twentiethcenturyfoxfilmcorporation duneentertainment lightstormentertainment samworthington zoesaldana sigourneyweaver release in the year 2009 lang en longMovie

Title: Serenity
Description: when the renegade crew of serenity agrees to hide a fugitive on their ship  they find themselves in an action-packed battle between the relentless military might of a totalitarian regime who will destroy anything – or anyone – to get the girl back and the bloodthirsty creatures who roam the uncharted areas of 

In [14]:
model.save_pretrained("sbert_model")  # Save the model

In [15]:
df.to_csv('../Data/vectors.csv',index=False)